# Import Packages 

In [1]:
import numpy as np
import re
import pandas as pd
from scipy.spatial import distance
import random
import os
import glob

# Generating random protein sequences

In [54]:
# Generate random protein sequence 

list_size = 50000 # sequence number 
random_seq = []
gen_name = []

for j in range(list_size):
    seq_n = "seq"+ str(j+1)
    gen_name.append(seq_n)
    # equal probability for all amino acid 
    weights = [0.05 for i in range(20)] 
    # generate sequence randomly with length = 256 aa
    x = ''.join(random.choices('ACDEFGHIKLMNPQRSTVWY', weights= weights, k= 256))
    random_seq.append(x) 

In [3]:
len(random_seq)

50000

In [13]:
# write fasta file 

seq_file = open("/home/defense/shokor/mapping2/data/random_seq.fasta", "w")

for i in range(len(random_seq)):

    seq_file.write(">" + gen_name[i] + "\n" + random_seq[i] + "\n")

seq_file.close()

# Generate mutated real sequence

In [3]:
def read_fasta(FASTA):
    '''
    Read fasta file 
    
    input: fasta file
    
    output: 
         - list of sequence  
         - list of sequence names
    '''
    
    m_file=open(FASTA,'r')

    data=''
    name_list=[]
    seq_list=[]

    for line in m_file:

        line=line.strip()
        for i in line:
            if i=='>':
                name_list.append(line[1:])
                if data :
                    seq_list.append(data)
                    data=''

                break
            else:
                line=line.upper()
        if all([k==k.upper() for k in line]):
            data=data+line
    if data:
        data = " ".join(data)
        seq_list.append(data)
        
    return seq_list, name_list


we choose uniprot sequence with 30% identity

In [1]:
# select sequences with 30% identity
!mmseqs easy-linclust /home/defense/shokor/mapping2/data/uni5033.fasta /home/defense/shokor/mapping2/data/id_30/uni5033 tmp --cov-mode 1 --min-seq-id 0.3

easy-linclust /home/defense/shokor/mapping2/data/uni5033.fasta /home/defense/shokor/mapping2/data/id_30/uni5033 tmp --cov-mode 1 --min-seq-id 0.3 

MMseqs Version:                     	7aade9df7475ae7c699b2074b5e4daa52e0245f1
Cluster mode                        	0
Max connected component depth       	1000
Similarity type                     	2
Threads                             	40
Compressed                          	0
Verbosity                           	3
Substitution matrix                 	nucl:nucleotide.out,aa:blosum62.out
Add backtrace                       	false
Alignment mode                      	0
Alignment mode                      	0
Allow wrapped scoring               	false
E-value threshold                   	0.001
Seq. id. threshold                  	0.3
Min alignment length                	0
Seq. id. mode                       	0
Alternative alignments              	0
Coverage threshold                  	0.8
Coverage mode                       	1
Max sequence lengt

In [4]:
# Read fasta file and generate 2 list of sequences and names
real_fasta_file = "/home/defense/shokor/mapping2/data/id_30/uni5033_rep_seq.fasta" 
real_seq_list, real_name = read_fasta(real_fasta_file) 

In [7]:
# select just the anme of a sequence (between ||)
ids = []
for n in real_name:
    ids.append(re.findall(r'\|(.*?)\|', n))
    
real_name_list = [item[0] for item in ids]

In [9]:
len(real_name_list)

1971

In [10]:
# Remove sequence longer than 256 aa
drop_out = []
for i, s in enumerate(real_seq_list):
    if len(s)>256:
        drop_out.append(i)
        
for index in sorted(drop_out, reverse=True):
    del real_seq_list[index]
    del real_name_list[index]
    
# len(real_seq_list)
# len(real_name_list)

In [11]:
def mutate(sequence_list, mutation_rate):
    ''' 
    Produce random mutations in a given list of sequence with specific percentage
    
    input:
        - list of sequences
        - mutation percentage
        
    output:
        - list of mutated sequence
    '''
    mutate_list = []
    # all amino acid have the same probability (1/20) to be chosen
    weights = [0.05 for i in range(20)] 

    for seq in sequence_list: 
        prot_list = list(seq)
        for i in range(len(seq)):
            r = random.random()
            if r < mutation_rate:
                mutation_site = random.randint(0, len(prot_list) - 1)
                
                prot_list[mutation_site] = random.choices(list('ACDEFGHIKLMNPQRSTVWY'), weights= weights)

        p = ''.join([item for sublist in prot_list for item in sublist])
        mutate_list.append(p)
    return mutate_list

In [12]:
# generate mutated sequecne with different percentage

mutation_rate_list = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99]

all_mutate_list = []
for mutation_rate in mutation_rate_list:
    mutate_list = mutate(real_seq_list, mutation_rate)
    all_mutate_list.append(mutate_list)


In [13]:
# Create a dictionary of mutated sequecne and their percentage
keys = ['mutate_10', 'mutate_20', 'mutate_30', 'mutate_40', 'mutate_50', 'mutate_60', 'mutate_70', 'mutate_80', 'mutate_90', 'mutate_95', 'mutate_99']
dictionary = dict(zip(keys, all_mutate_list))

In [14]:
# select the
all_names = []
for key in keys:
    
    for names in real_name_list: 
        seq_n = names + ' ' + key
        all_names.append(seq_n)
    
len(all_names)

4477

In [16]:
# save all sequence in the one list 
seq_mutate_list= [item for s in all_mutate_list for item in s]
len(seq_mutate_list)

4477

In [17]:
# write fasta file 

seq_file = open("/home/defense/shokor/mapping2/data/mutate_real_seq.fasta", "w")

for i in range(len(seq_mutate_list)):

    seq_file.write(">" + all_names[i] + "\n" + seq_mutate_list[i] + "\n")

seq_file.close()

In [16]:
# mutate_fasta_file = "/home/defense/shokor/mapping2/data/mutate_real_seq.fasta"
# mutate_seq_list, mutate_name = read_fasta(mutate_fasta_file)
# len(mutate_seq_list)

### Blosum62

https://www.biotite-python.org/examples/gallery/sequence/blosum_dendrogram.html

In [18]:
!pip install biotite

In [20]:
import biotite.sequence.align as align
import biotite.sequence as seq

# Load Blossum62 matrix form biotite package
matrix = align.SubstitutionMatrix.std_protein_matrix()

In [35]:
# Matrix should not contain ambiguous symbols or stop signal
matrix = align.SubstitutionMatrix(
    seq.Alphabet(matrix.get_alphabet1().get_symbols()[:-4]),
    seq.Alphabet(matrix.get_alphabet2().get_symbols()[:-4]),
    matrix.score_matrix()[:-4, :-4]
)
similarities = matrix.score_matrix()
print(matrix)

    A   C   D   E   F   G   H   I   K   L   M   N   P   Q   R   S   T   V   W   Y
A   4   0  -2  -1  -2   0  -2  -1  -1  -1  -1  -2  -1  -1  -1   1   0   0  -3  -2
C   0   9  -3  -4  -2  -3  -3  -1  -3  -1  -1  -3  -3  -3  -3  -1  -1  -1  -2  -2
D  -2  -3   6   2  -3  -1  -1  -3  -1  -4  -3   1  -1   0  -2   0  -1  -3  -4  -3
E  -1  -4   2   5  -3  -2   0  -3   1  -3  -2   0  -1   2   0   0  -1  -2  -3  -2
F  -2  -2  -3  -3   6  -3  -1   0  -3   0   0  -3  -4  -3  -3  -2  -2  -1   1   3
G   0  -3  -1  -2  -3   6  -2  -4  -2  -4  -3   0  -2  -2  -2   0  -2  -3  -2  -3
H  -2  -3  -1   0  -1  -2   8  -3  -1  -3  -2   1  -2   0   0  -1  -2  -3  -2   2
I  -1  -1  -3  -3   0  -4  -3   4  -3   2   1  -3  -3  -3  -3  -2  -1   3  -3  -1
K  -1  -3  -1   1  -3  -2  -1  -3   5  -2  -1   0  -1   1   2   0  -1  -2  -3  -2
L  -1  -1  -4  -3   0  -4  -3   2  -2   4   2  -3  -3  -2  -2  -2  -1   1  -2  -1
M  -1  -1  -3  -2   0  -3  -2   1  -1   2   5  -2  -2   0  -1  -1  -1   1  -1  -1
N  -2  -3   1   

In [23]:
from scipy.special import softmax

In [42]:
# transfom score into probability
trans_score = softmax(similarities, axis=0)

In [52]:
# generate a dataframe of substitution probability 
blosum_df = pd.DataFrame(data=trans_score, index=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y'], columns=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y'])

In [ ]:
def mutate(sequence_list, mutation_rate):
    '''
    Produce mutations based on blossum62 substitution matrix in a given list of sequence with specific percentage
    
    input:
        - list of sequences
        - mutation percentage
        
    output:
        - list of mutated sequence
    '''
    mutate_list = []
    
    aa = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']

    for seq in sequence_list: 
        prot_list = list(seq)
        for i in range(len(seq)):
            r = random.random()
            
            if r < 0.1:
                mutation_site = random.randint(0, len(prot_list) - 1)

                if prot_list[mutation_site] in aa:
                    weights = blosum_df[prot_list[mutation_site]].values.tolist()
            
                    if isinstance(weights[0], list):
                        new_weights = []
                        for n in weights:
                            new_weights = new_weights + n
                    else:
                        new_weights = weights 

                    prot_list[mutation_site] = random.choices(list('ACDEFGHIKLMNPQRSTVWY'), weights= new_weights)
                  
        p = ''.join([item for sublist in prot_list for item in sublist])
        mutate_list.append(p)
    return mutate_list 

In [ ]:
# generate mutated sequecne with different percentage

mutation_rate_list = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99]


all_blosmutate_list = []
for mutation_rate in mutation_rate_list:
    mutate_list = mutate(real_seq_list, mutation_rate)
    all_blosmutate_list.append(mutate_list)

In [ ]:
# save all sequence in the one list 
seq_blosmutate_list= [item for s in all_blosmutate_list for item in s]
len(seq_blosmutate_list)

In [ ]:
# write fasta file 

seq_file = open("/home/defense/shokor/mapping2/data/blosum_mutate_real_seq.fasta", "w")

for i in range(len(all_blosmutate_list)):

    seq_file.write(">" + all_names[i] + "\n" + seq_blosmutate_list[i] + "\n")

seq_file.close()

# Generating protein sequences with expected prob

Composition in percent for the complete database:

   Ala (A) 8.25   Gln (Q) 3.93   Leu (L) 9.65   Ser (S) 6.64

   Arg (R) 5.53   Glu (E) 6.72   Lys (K) 5.80   Thr (T) 5.35

   Asn (N) 4.06   Gly (G) 7.07   Met (M) 2.41   Trp (W) 1.10

   Asp (D) 5.46   His (H) 2.27   Phe (F) 3.86   Tyr (Y) 2.92

   Cys (C) 1.38   Ile (I) 5.91   Pro (P) 4.74   Val (V) 6.86



   Asx (B) 0.000  Glx (Z) 0.000  Xaa (X) 0.00

In [36]:
# Generate protein sequences with specific prob

list_size = 50000 # sequence number 
expected_seq = []
seq_name = []
weights = [0.0825, 0.0138, 0.0546, 0.0672, 0.0386, 0.0707, 0.0227, 0.0591, 0.058, 0.0965, 0.0241, 0.0406, 0.0474, 0.0393, 0.0553, 0.0664, 0.0535, 0.0686, 0.011, 0.0292]

for j in range(list_size):
    seq_n = "expected_seq"+ str(j+1)
    seq_name.append(seq_n)
    y = ''.join(random.choices('ACDEFGHIKLMNPQRSTVWY', weights= weights, k= 256))
    expected_seq.append(y) 

In [37]:
# write fasta file 

expected_seq_file = open("/home/defense/shokor/mapping2/data/id_30/expected_seq.fasta", "w")

for i in range(len(expected_seq)):

    expected_seq_file.write(">" + seq_name[i] + "\n" + expected_seq[i] + "\n")

expected_seq_file.close()

In [38]:
# expected_fasta_file = "/home/defense/shokor/mapping2/data/expected_seq.fasta"
# expected_seq_list, expected_name = read_fasta(expected_fasta_file) 

# MMseqs2

## Install Swiss prot

In [49]:
# !mkdir /home/defense/shokor/mapping2/data/databases
# !mkdir /home/defense/shokor/mapping2/data/databases/uniprot

In [48]:
!mmseqs databases UniProtKB/Swiss-Prot /home/defense/shokor/mapping2/data/databases/uniprot/swissprot tmp 

databases UniProtKB/Swiss-Prot /home/defense/shokor/mapping2/data/databases/uniprot/swissprot tmp 

MMseqs Version:              	13.45111
Force restart with latest tmp	false
Remove temporary files       	false
Compressed                   	0
Threads                      	40
Verbosity                    	3

createdb tmp/16198825224019552537/uniprot_sprot.fasta.gz /home/defense/shokor/mapping2/data/databases/uniprot/swissprot --compressed 0 -v 3 

Converting sequences
[565904] 2s 353ms
Time for merging to swissprot_h: 0h 0m 0s 121ms
Time for merging to swissprot: 0h 0m 0s 241ms
Database type: Aminoacid
Time for processing: 0h 0m 3s 689ms
prefixid /home/defense/shokor/mapping2/data/databases/uniprot/swissprot_h tmp/16198825224019552537/header_pref.tsv --tsv --threads 40 -v 3 

[=================================================================] 100.00% 565.93K 0s 175ms          ] 23.56% 133.32K eta 0s       
Time for merging to header_pref.tsv: 0h 0m 0s 235ms
Time for processing: 0h 0m 0s

## Search for generated sequences in swiss prot database

In [52]:
# !mkdir /home/defense/shokor/mapping2/data/alignement

In [51]:
# Random sequences
!mmseqs easy-search /home/defense/shokor/mapping2/data/random_seq.fasta /home/defense/shokor/mapping2/data/databases/uniprot/swissprot /home/defense/shokor/mapping2/data/alignement/random_search.m8 tmp

easy-search /home/defense/shokor/mapping2/data/random_seq.fasta /home/defense/shokor/mapping2/data/databases/uniprot/swissprot /home/defense/shokor/mapping2/data/alignement/random_search.m8 tmp 

MMseqs Version:                        	13.45111
Substitution matrix                    	nucl:nucleotide.out,aa:blosum62.out
Add backtrace                          	false
Alignment mode                         	3
Alignment mode                         	0
Allow wrapped scoring                  	false
E-value threshold                      	0.001
Seq. id. threshold                     	0
Min alignment length                   	0
Seq. id. mode                          	0
Alternative alignments                 	0
Coverage threshold                     	0
Coverage mode                          	0
Max sequence length                    	65535
Compositional bias                     	1
Max reject                             	2147483647
Max accept                             	2147483647
Include identic

In [62]:
# print first column of search file
!cat /home/defense/shokor/mapping2/data/alignement/random_search.m8 | sed 's/|/ /' | awk '{print $1}'


seq11648
seq1885
seq24400
seq19982
seq39583
seq22586
seq19626


These sequences will be removed 

In [59]:
# Expected sequences
!mmseqs easy-search /home/defense/shokor/mapping2/data/expected_seq.fasta /home/defense/shokor/mapping2/data/databases/uniprot/swissprot /home/defense/shokor/mapping2/data/alignement/expected_search.m8 tmp

easy-search /home/defense/shokor/mapping2/data/expected_seq.fasta /home/defense/shokor/mapping2/data/databases/uniprot/swissprot /home/defense/shokor/mapping2/data/alignement/expected_search.m8 tmp 

MMseqs Version:                        	13.45111
Substitution matrix                    	nucl:nucleotide.out,aa:blosum62.out
Add backtrace                          	false
Alignment mode                         	3
Alignment mode                         	0
Allow wrapped scoring                  	false
E-value threshold                      	0.001
Seq. id. threshold                     	0
Min alignment length                   	0
Seq. id. mode                          	0
Alternative alignments                 	0
Coverage threshold                     	0
Coverage mode                          	0
Max sequence length                    	65535
Compositional bias                     	1
Max reject                             	2147483647
Max accept                             	2147483647
Include ide

In [61]:
# print first column of search file
!cat /home/defense/shokor/mapping2/data/alignement/expected_search.m8 | sed 's/|/ /' | awk '{print $1}'

expected_seq33114
expected_seq16872
expected_seq38982
expected_seq38982
expected_seq42840


These sequences will be removed 

## Choose the sequences with 30% identity 

In [64]:
!mkdir /home/defense/shokor/mapping2/data/id_30

### Random sequences 

In [65]:
!mmseqs easy-linclust /home/defense/shokor/mapping2/data/random_seq.fasta /home/defense/shokor/mapping2/data/id_30/randclust tmp --cov-mode 1 --min-seq-id 0.3

easy-linclust /home/defense/shokor/mapping2/data/random_seq.fasta /home/defense/shokor/mapping2/data/id_30/randclust tmp --cov-mode 1 --min-seq-id 0.3 

MMseqs Version:                     	13.45111
Cluster mode                        	0
Max connected component depth       	1000
Similarity type                     	2
Threads                             	40
Compressed                          	0
Verbosity                           	3
Substitution matrix                 	nucl:nucleotide.out,aa:blosum62.out
Add backtrace                       	false
Alignment mode                      	0
Alignment mode                      	0
Allow wrapped scoring               	false
E-value threshold                   	0.001
Seq. id. threshold                  	0.3
Min alignment length                	0
Seq. id. mode                       	0
Alternative alignments              	0
Coverage threshold                  	0.8
Coverage mode                       	1
Max sequence length                 	65535
Co

### Expected sequences

In [68]:
!mmseqs easy-linclust /home/defense/shokor/mapping2/data/expected_seq.fasta /home/defense/shokor/mapping2/data/id_30/expclust tmp --cov-mode 1 --min-seq-id 0.3

easy-linclust /home/defense/shokor/mapping2/data/expected_seq.fasta /home/defense/shokor/mapping2/data/id_30/expclust tmp --cov-mode 1 --min-seq-id 0.3 

MMseqs Version:                     	13.45111
Cluster mode                        	0
Max connected component depth       	1000
Similarity type                     	2
Threads                             	40
Compressed                          	0
Verbosity                           	3
Substitution matrix                 	nucl:nucleotide.out,aa:blosum62.out
Add backtrace                       	false
Alignment mode                      	0
Alignment mode                      	0
Allow wrapped scoring               	false
E-value threshold                   	0.001
Seq. id. threshold                  	0.3
Min alignment length                	0
Seq. id. mode                       	0
Alternative alignments              	0
Coverage threshold                  	0.8
Coverage mode                       	1
Max sequence length                 	65535
C

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

